### This file will use the Rotational Forest Classifier from the Aeon library

In [ ]:
import _Data_Processing
from importlib import reload

'''
this function returns 7 objects:
	data: sets of data as X_train, X_val, X_ind, y_train, y_val, y_ind
	feature_list: list of all feature names with a column index (dict)
'''

reload(_Data_Processing)

lstm_format = False

X, X_train, X_val, X_ind,\
y, y_train, y_val, y_ind,\
feature_subsets, scaler =\
_Data_Processing.preprocess_data(
	file_name   =		'spx_ndx_full.csv'
	,indp_size  =		0.01
	,test_size  =		0.20
	,shfl_splt	=		False
	,t_start    =		570
	,t_end		=		720
	,mod_type	=		'Area_Classification'
	,target_t	=		30
	,num_class	=		2
	,split_val	=		5
	,verbose	=		1
	,scaler		=		'Standard'
	,frmt_lstm	=		lstm_format
	,time_steps =		5
	,keep_price =		True
	,optm_data	=		True
)

In [ ]:
import joblib
joblib.dump(scaler, 'scaler/tmp.joblib')

In [ ]:
import _Data_Rotating
from importlib import reload
'''
This function returns 4 items.
-	2D array of X partitions, by featurespace and samplespace
-	1D array of index-lists for each featurespace-partition specific features
-	1D array of featurespace-partition specific rotation transformer functions
-	1D array of y partitions for each samplespace partition.
'''

reload(_Data_Rotating)

X_train_parts, X_find_parts, X_trans_parts, y_parts = _Data_Rotating.rotate_partitions(
	X				=	X_train
	,y				=	y_train
	,n_feat_parts	=	8
	,feat_subsets	=	feature_subsets
	,feat_part_type	=	'by_subset'
	,fraction_feats	=	0.75
	,no_feat_overlap=	False
	,feats_for_all	=	[]
	,rotation_type	=	'PCA'
	,rotation_filter=	False
	,filter_type	=	'Retention'
	,filter_value	=	0.95
	,n_sample_parts	=	1
	,smpl_part_type	=	'Even'
	,sample_overlap	=	0.9
	,sample_shuffle	=	False
	,lstm_format	=	lstm_format
)

'''
all partitions
	X_train_parts
all sample partitions of THE featurespace partition i
	X_train_parts[i]
The single partition of THE samplespace partition j of THE featurespace partition i
	X_train_parts[i][j] -- -- -- This is now a set of trainable data
'''

In [ ]:
for featureset in feature_subsets:
	print(featureset)

In [ ]:
import _Modelset_Training
reload(_Modelset_Training)

_Modelset_Training.show_available_model_types()

In [ ]:
import _Modelset_Training
from importlib import reload

'''
Function returns a 3D list of trained models that are fit to the resepective dataset.
	Dim1:	Featurespace, here are the subsetctions/partitions for each unique set of features
	Dim2:	Samplespace, here are the subsetctions/partitions for each unique set of samples
	Dim3:	Modelspace, here are the individual models trained on a unique partition
'''
reload(_Modelset_Training)

models = _Modelset_Training.train_models(
	model_types	=	['nn']
	,data_parts	=	X_train_parts
	,findx_parts=	X_find_parts
	,trans_parts=	X_trans_parts
	,trgt_parts	=	y_parts
	,X_valid	=	X_val
	,y_valid	=	y_val
	,param_mode	=	'custom'
	,cst_mod_prm=	[{
		'optimizer_type'	:	'Adam'
		,'optimizer_kwarg':{
			'learning_rate'	:	0.0005
			#,'momentum'		:	0.9
			}
		,'time_steps'		:	5
		,'LSTM'				:	False
		,'shuffle_train'	:	True
		,'epochs'			:	5
		,'rlr_patience'		:	20
		,'architecture'	:	'default_deep'
		#,'external_cw'	:	_Utility.get_class_weights(y_train)
		}]
	,tnr_verbose=	True
	,lstm_frmt	=	lstm_format
	,use_cls_wt	=	True
)

In [ ]:
import _Model_Evaluation
import _Neural_Net

reload(_Model_Evaluation)
reload(_Neural_Net)

_Model_Evaluation.evaluate_models(
		models=models
		,X_findx=X_find_parts
		,X_trans=X_trans_parts
		,X_train=X_train_parts
		,y_train=y_parts
		,X_test=X_val
		,y_test=y_val
		,prfm_gnrl	=	'all'
		,prfm_stat	=	'all'
		,disp_mthd	=	'as_value'
		,test_whch	=	'independent'
		,pred_type	=	'classification'
		,lstm_frmt	=	lstm_format
)

In [ ]:
import _Metamodel
from importlib import reload
import _Utility

reload(_Metamodel)
metamodel, prediction_set = _Metamodel.train_test_meta_model(
	models		=	models
	,X_findx	=	X_find_parts
	,X_trans	=	X_trans_parts
	,X_test		=	X_val
	,y_test		=	y_val
	,val_size	=	0.2
	,shuffle	=	True
	,metam_type =	'NN'
	,use_cls_wt =	True
	,use_mm_params=	True
	,metam_params={
		'optimizer_type':	'Adam'
		,'optimizer_kwarg':{
			'learning_rate'	:	0.001
			#,'momentum':0.9
			}
		,'time_steps'		:	5
		,'LSTM'				:	False
		,'shuffle_train'	:	True
		,'epochs'			:	75
		,'batch_size'		:	30
		,'rlr_patience'		:	10
		,'architecture'	:	'default_deep'
		#,'custom_val_data'	:	(X_train, y_train)
		}
)

final connective test will be:
-	casually train fair/good metamodel
-	load into a master model
-	predict on X_ind
-	score model
-	model performs at same level
-	save model
-	load same model
-	test on X_ind again
-	will have indentical score to pre save/load

In [ ]:
import _Data_Processing
from importlib import reload
reload(_Data_Processing)
lstm_format = False
X_tall, X_test, _, __,\
y_tall, y_test, ___, ____,\
feature_subsets =\
_Data_Processing.preprocess_data(
	file_name   =		'betaset_tmp.csv'
	,indp_size  =		0.01
	,test_size  =		0.01
	,shfl_splt	=		False
	,t_start    =		645
	,t_end		=		900
	,mod_type	=		'Classification'
	,target_t	=		5
	,num_class	=		2
	,split_val	=		5
	,verbose	=		1
	,scaler		=		'Standard'
	,frmt_lstm	=		lstm_format
	,time_steps =		5
	,keep_price =		False
)

In [ ]:
i = [0,5]

print(i[0])
print(i[1])

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import _Master_Model
reload(_Master_Model)
from importlib import reload

master = _Master_Model.Master(
	model_depth		=	2
	,all_models 	=	[models, metamodel]
	,lvl0_formatters=	[X_find_parts, X_trans_parts]
)

#	train	val		ind

test_on_X = X_ind
test_on_y = y_ind

m_pred = master.master_predict(test_on_X, threshold=0.5)
print(accuracy_score(test_on_y, m_pred))

#master.master_predict_fullth(test_on_X, test_on_y, definition='min')

#print((m_pred[0]))

#Create the confusion matrix
cm = confusion_matrix(test_on_y, m_pred)
# Plot the confusion matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', \
			xticklabels=range(2), yticklabels=range(2))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix for Meta-Model Independent Test')
plt.show()

In [ ]:
master.save_model()

In [ ]:
import _Data_Processing
from importlib import reload
import joblib
reload(_Data_Processing)
lstm_format = False
X_test, _, _, __,\
y_test, _, ___, ____,\
feature_subsets, scaler =\
_Data_Processing.preprocess_data(
	file_name   =		'pricearea_test.csv'
	,indp_size  =		0.01
	,test_size  =		0.01
	,shfl_splt	=		False
	,t_start    =		645
	,t_end		=		800
	,mod_type	=		'Area_Classification'
	,target_t	=		45
	,num_class	=		2
	,split_val	=		5
	,verbose	=		1
	,scaler		=		'Custom'
	,cstm_scale	=		joblib.load('scaler/tmp.joblib')
	,frmt_lstm	=		lstm_format
	,time_steps =		5
	,keep_price =		False
	,indices	=		0
)

from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import _Master_Model
from importlib import reload
reload(_Master_Model)

test_on_X = X_test
test_on_y = y_test

loadmodel = _Master_Model.Master(
	model_depth=2
)
loadmodel.load_model('pred1_63p2_acc-645-800')
m_pred = loadmodel.master_predict(test_on_X, threshold=0.5)
print(accuracy_score(test_on_y, m_pred))

#Create the confusion matrix
cm = confusion_matrix(test_on_y, m_pred)
# Plot the confusion matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', \
			xticklabels=range(2), yticklabels=range(2))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix for Meta-Model Independent Test')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

import _Data_Processing
from importlib import reload
import joblib
reload(_Data_Processing)
lstm_format = False
X_set, _, _, __,\
y_set, _, ___, ____,\
feature_subsets, scaler =\
_Data_Processing.preprocess_data(
	file_name   =		'pricearea_c.csv'
	,indp_size  =		0.01
	,test_size  =		0.01
	,shfl_splt	=		False
	,t_start    =		645
	,t_end		=		800
	,mod_type	=		'Area_Classification'
	,target_t	=		45
	,num_class	=		2
	,split_val	=		5
	,verbose	=		0
	,scaler		=		'None'
	,cstm_scale	=		joblib.load('scaler/tmp.joblib')
	,frmt_lstm	=		lstm_format
	,time_steps =		5
	,keep_price =		True
	,indices	=		0
)


In [ ]:
cm_vals = []
for i in range(len(m_pred)):
	if(test_on_y[i] == 0):
		if(m_pred[i] == 0):
			cm_vals.append(0)
		if(m_pred[i] == 1):
			cm_vals.append(1)
	if(test_on_y[i] == 1):
		if(m_pred[i] == 0):
			cm_vals.append(2)
		if(m_pred[i] == 1):
			cm_vals.append(3)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

df = pd.DataFrame(X_set)
df['score'] = cm_vals
df['mpred'] = m_pred
df['target'] = y_set
#df = df.drop(df[df['score']%2==0].index)

pd.set_option('display.max_rows',None)
co = df.corr()['target']
print(co.sort_values())

In [ ]:
from sklearn.svm import SVC

df_pair = pd.DataFrame()
df_pair['ft'] = df[503].values
df_pair['model_prediction'] = m_pred

X_svm = df_pair.values
y_svm = df['target'].values

clf = SVC(kernel='linear',C=1.0).fit(X_svm, y_svm)

y_svmpred = clf.predict(X_svm)

#Create the confusion matrix
cm = confusion_matrix(y_test, y_svmpred)
# Plot the confusion matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', \
			xticklabels=range(2), yticklabels=range(2))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'asdfasdfasdfasdf')
plt.show()

In [ ]:
import mplfinance as mpf
import pandas as pd
import numpy as np


print(int(len(X_test[:,0])/155))

num_candles = 155

for section in range(int(len(X_test[:,0])/num_candles)):
	section*=num_candles
	section_end = section+num_candles
	X_thold = X_test[section:section_end,:]

	#custom coloring
	color_map = {
		0: 'white',
		1: 'red',
		2: 'white',
		3: 'green'
	}
	colors = [color_map[condition] for condition in cm_vals[section:section_end]]
	mc = mpf.make_marketcolors(up='g',down='r')
	custom_style = mpf.make_mpf_style(marketcolors=mc)#, gridcolor='lightgray')

	h = X_thold[:,0]
	l = X_thold[:,1]
	c = X_thold[:,2]
	o = np.roll(X_thold[:,2], shift=1)

	#o,c swapping function for appropriate coloring
	for i in range(len(c)):
		if(cm_vals[section+i]%2==1):
		#	if(c[i]<o[i]):#force green
			c[i],o[i] = h[i],l[i]
			#else:
		#		pass#is green
		else:#force red
			#if(c[i]>o[i]):
			c[i],o[i] = l[i],h[i]

	data = {
		'Date':range(0,len(X_thold[:])*1000000000,1000000000),
		'Open':o,
		'High':h,
		'Low':l,
		'Close':c
	}
	df = pd.DataFrame(data)
	#df['color'] = colors
	df['Date'] = pd.to_datetime(df['Date'])
	df.set_index('Date',inplace=True)

	df_NULL = pd.DataFrame

	mpf.plot(df.iloc[10:-10,:], type='candle',style=custom_style,figratio=(20,8))
	#mpf.plot(df, addplot=plot,style=custom_style,figratio=(20,8))

In [ ]:
import _Utility
from sklearn.metrics import accuracy_score
y_pred = _Utility.dummy_predict(X_test=X_train_parts[0][0] , prediction=1)
print(accuracy_score(y_parts[0], y_pred))